In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService

QiskitRuntimeService.save_account(
    channel="ibm_quantum",
    token="46c0ed7e272dd2850d2c179a39a994a3659485d3ada7df19316fb540fe4879eb978acd183540cac705d740095274367104c0ca7d62d0ea835f382735f8b2b159",
    set_as_default=True,
    overwrite=True,
)

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.circuit.library import EfficientSU2
import numpy as np

# Load IBM Quantum service
service = QiskitRuntimeService()

# Select the least busy backend (real quantum device, not simulator)
backend = service.least_busy(operational=True, simulator=False)
print(f"Selected backend: {backend.name}")
circuit = EfficientSU2(127, entanglement="linear", flatten=True)
circuit.measure_all()
# The circuit is parametrized, so we will define the parameter values for execution
param_values = np.random.rand(circuit.num_parameters)
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circuit = pm.run(circuit)
print(f">>> Circuit ops (ISA): {isa_circuit.count_ops()}")

# Saber Parameters
q = 2**13  # Large modulus
p = 2**10  # Smaller modulus
T = 2      # Threshold for rounding

# Quantum random number generator using Qiskit Runtime
def quantum_random_int(bits=8):
    """Generates a random integer using a quantum circuit and Qiskit Runtime."""
    qc = QuantumCircuit(bits, bits)
    qc.h(range(bits))  # Apply Hadamard gates
    qc.measure(range(bits), range(bits))

    # Transpile circuit for the backend
    qc_transpiled = transpile(qc, backend)

    print("Submitting the job...")  
    sampler = Sampler(mode=backend)
    job = sampler.run([(isa_circuit, param_values)])
    print(f">>> Job ID: {job.job_id()}")
    print(f">>> Job Status: {job.status()}") 
    result = job.result()
    pub_result = result[0]  
    counts = pub_result.data.meas.get_counts()
    
    # Convert quantum measurements to integer
    most_common_bitstring = max(counts, key=counts.get)  # Get most common output
    random_value = int(most_common_bitstring, 2)  # Convert to integer

    return random_value % 256 
 
# Saber Helper Functions
def gen(seed):
    np.random.seed(seed)
    return np.random.randint(0, q, (2, 2))

def beta_mu(shape, seed):
    np.random.seed(seed)
    return np.random.randint(-T, T+1, shape)

def round_mod_q_to_p(x):
    return np.round ((x * p // q)) % p

# Saber KeyGen
def keygen():
    seedA = quantum_random_int(8)
    A = gen(seedA)
    seedR = quantum_random_int(8)
    s = beta_mu((2, 1), seedR)
    h = np.full((2, 1), q // 2)
    b = round_mod_q_to_p((A.T @ s + h) % q)
    return (seedA, b), s

# Saber Encryption
def encrypt(pk, m):
    seedA, b = pk
    A = gen(seedA)
    seedE = quantum_random_int(8)
    s = beta_mu((2, 1), seedE)
    h1 = np.full((2, 1), q // 2)
    u = round_mod_q_to_p((A @ s + h1) % q)
    v = ((b.T @ (s % p)) % p).flatten()
    v = np.tile(v, len(m))
    h2 = p // 2
    c = (v + h2 + (m * (p // 2))) % p
    return u, c

# Saber Decryption
def decrypt(sk, c):
    u, v = c
    h2 = p // 2
    v_prime = ((u.T @ (sk % p)) % p).flatten()
    m = ((v - v_prime + h2) % p) // (p // 2)
    return m.astype(int)

# Run Saber KeyGen
pk, sk = keygen()
m = np.array([1,1,0,1,1,0,0,1,0,1,1,1,0,0,1,1,1,0,1,0,0])
ciphertext = encrypt(pk, m)
decrypted_m = decrypt(sk, ciphertext)
print("Original Message:", m)
print("Encrypted Message:", ciphertext)
print("Decrypted Message:", decrypted_m)


Selected backend: ibm_kyiv
>>> Circuit ops (ISA): OrderedDict({'rz': 6391, 'sx': 4303, 'ecr': 1749, 'measure': 127, 'x': 49, 'barrier': 1})
Submitting the job...
>>> Job ID: cz3zby710wx0008bc6a0
>>> Job Status: QUEUED
Submitting the job...
>>> Job ID: cz3zc7gh0kc00088s8qg
>>> Job Status: QUEUED
Submitting the job...
>>> Job ID: cz3zcb9p6030008cbba0
>>> Job Status: QUEUED
Original Message: [1 1 0 1 1 0 0 1 0 1 1 1 0 0 1 1 1 0 1 0 0]
Encrypted Message: (array([[788],
       [739]]), array([ 495,  495, 1007,  495,  495, 1007, 1007,  495, 1007,  495,  495,
        495, 1007, 1007,  495,  495,  495, 1007,  495, 1007, 1007]))
Decrypted Message: [0 0 1 0 0 1 1 0 1 0 0 0 1 1 0 0 0 1 0 1 1]
